In [1]:
import pandas as pd
import numpy as np
from tkinter import filedialog
import tkinter as tk
import openpyxl
import customtkinter as ctk
from customtkinter import filedialog
import funciones


### Subir el archivo y guardarlo en un dataframe

In [16]:
archivo = None
while True:
    archivo = funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        try:
            padon_1 = pd.read_excel(ruta_archivo, engine='openpyxl')
            #padron = pd.read_csv(archivo, sep=";", encoding="ISO-8859-1")
            print(padon_1)
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")

<_io.TextIOWrapper name='G:/Mi unidad/osoetsylra/importaciones_TIS_SD_ETC_a_sidefi/meplife/junio/UOETSYLRA JUNIO.xlsx' mode='r' encoding='cp1252'>
      Codigo  CUIT_Empleador  CUIL_Titular  Parentesco  CUIL_Familiar   
0     127406     20240830214   20240830214           0    20240830214  \
1     127406     20119590958   20119590958           0    20119590958   
2     127406     27425694192   27425694192           0    27425694192   
3     127406     27425694192   27425694192           3    20549159851   
4     127406     27270992132   27270992132           0    27270992132   
...      ...             ...           ...         ...            ...   
1682  127406     20190072577   20190072577           0    20190072577   
1683  127406     27321459590   27321459590           0    27321459590   
1684  127406     27174774221   27174774221           0    27174774221   
1685  127406     27174774221   27174774221           1    20149595150   
1686  127406     27316755637   27316755637        

### duplicados

In [17]:
padon_1.isnull().sum()

Codigo                  0
CUIT_Empleador          0
CUIL_Titular            0
Parentesco              0
CUIL_Familiar           0
Tipo_Doc                0
Nro_Doc                 0
Apellido                0
Nombre                  0
Sexo                    0
Estado_Civil            0
Fch_Nac                 0
Nacionalidad            0
Calle                   0
Nro                  1484
Piso                 1667
Dpto                 1670
Localidad               0
CP                      0
Provincia               0
Tipo_domi               0
Telefono             1470
Revista                 0
Incapacidad             0
Tipo_Beneficiario       0
Fch_Alta_OS             0
nro_gpo                 0
nro_sss                 5
dtype: int64